In [6]:
import numpy as np
from PIL import Image
import porespy as ps
import openpnm as op
import os
import warnings
from tqdm import tqdm  # 导入 tqdm

# 抑制所有警告
warnings.filterwarnings("ignore")

np.random.seed(9)

# 定义文件夹数量和每个文件夹中的图像数量
num_folders = 10
images_per_folder = 100

# 定义voxel大小
voxel_size = 5.345e-6

throat_surface_tension = 0.072  # 假设水在水中的表面张力，单位为 J/m²


# 打开输出文件
with open('results.txt', 'w') as f:
    for folder_number in range(1, num_folders + 1):
        folder_name = str(folder_number)
        for i in range(images_per_folder):
            image_file = os.path.join(folder_name, f'image_{(folder_number-1)*images_per_folder + i}.png')
            try:
                # 读取并处理图像
                im = Image.open(image_file).convert('L')
                im_array = np.array(im)

                # 将图像转换为二进制格式（孔隙和骨架）
                binary_im = im_array < 128
                binary_im = binary_im.astype(int)

                # 计算孔隙率
                total_pores = np.prod(binary_im.shape)  # 计算总的像素数（即总孔隙数）
                open_pores = np.sum(binary_im)  # 计算非零像素数（即开放孔隙数）
                porosity = open_pores / total_pores  # 计算孔隙率
                #print(f"Porosity of the network is {porosity:.4f}")

                # 使用snow算法进行分割
                snow = ps.filters.snow_partitioning(binary_im)

                # 生成孔隙网络
                net1 = ps.networks.snow2(binary_im, voxel_size=voxel_size)
                pn = op.io.network_from_porespy(net1.network)

                # 健康检查并修复孤立节点
                h = op.utils.check_network_health(pn)
                if len(h['disconnected_pores']) > 0:
                    op.topotools.trim(network=pn, pores=h['disconnected_pores'])

                # 添加缺失的属性
                pn['pore.diameter'] = pn['pore.equivalent_diameter']
                pn['throat.diameter'] = pn['throat.inscribed_diameter']
                pn['throat.spacing'] = pn['throat.total_length']

                # 设置喉道表面张力
                pn['throat.surface_tension'] = throat_surface_tension

                # 禁用接触角模型
                pn['throat.contact_angle'] = 30.0  # 或者设置为一个常数，例如 0.0 度

                # 添加孔隙率到网络属性中
                pn['pore.porosity'] = porosity

                # 添加几何和物理模型
                pn.add_model(propname='throat.hydraulic_size_factors',
                             model=op.models.geometry.hydraulic_size_factors.pyramids_and_cuboids)
                pn.add_model(propname='throat.diffusive_size_factors',
                             model=op.models.geometry.diffusive_size_factors.pyramids_and_cuboids)
                pn.regenerate_models()

                # 添加几何模型并生成相关属性
                geo = op.models.collections.geometry.spheres_and_cylinders
                pn.add_model_collection(geo)
                pn.regenerate_models()

                h = op.utils.check_network_health(pn)

                gas = op.phase.Phase(network=pn)
                gas['pore.diffusivity'] = 1.0
                gas['pore.viscosity'] = 1.0
                gas.add_model_collection(op.models.collections.physics.basic)
                gas.regenerate_models()

                binary_im_3d = np.expand_dims(binary_im, axis=2)
                shape = binary_im_3d.shape
                spacing = voxel_size

                # 计算Formation factor
                fd = op.algorithms.FickianDiffusion(network=pn, phase=gas)
                fd.set_value_BC(pores=pn.pores('xmin'), values=1.0)
                fd.set_value_BC(pores=pn.pores('xmax'), values=0.0)
                fd.run()
                dC = 1.0
                L = (shape[0] + 6) * voxel_size
                A = shape[1] * shape[2] * voxel_size**2
                Deff = fd.rate(pores=pn.pores('xmin')) * (L / A) / dC
                F = 1 / Deff

                # 计算Permeability coefficient
                sf = op.algorithms.StokesFlow(network=pn, phase=gas)
                sf.set_value_BC(pores=pn.pores('xmin'), values=1.0)
                sf.set_value_BC(pores=pn.pores('xmax'), values=0.0)
                sf.run()
                dP = 1.0
                L = (shape[0] + 6) * voxel_size
                A = shape[1] * shape[2] * voxel_size**2
                K = sf.rate(pores=pn.pores('xmin')) * (L / A) / dP * 1e12

                # 保存结果到文件
                f.write(f'image_{(folder_number-1)*images_per_folder + i}.png,{porosity:.4f},{K[0]},{F[0]}\n')

                print(f'Processed {image_file}: Porosity = {porosity:.4f}, Permeability = {K[0]:.4f} Darcy, Formation factor = {F[0]:.4f}')

            except Exception as e:
                print(f'Error processing {image_file}: {e}')
                f.write(f'{image_file},Error:{e}\n')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Extracting pore and throat properties:   0%|          | 0/203 [00:00<?, ?it/s]

Processed 10\image_999.png: Porosity = 0.3366, Permeability = 0.4072 Darcy, Formation factor = 27.2587
